In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from numpy.random import RandomState

import scipy
from sklearn.cluster import KMeans




rs = RandomState(12345)

# import data
# make predicitions on this data set
predict_df = pd.read_csv('./synthetic_dataset_evaluation.csv')
predict_df.describe()

,Feature_A,Feature_C,Feature_E,Feature_K,Feature_L,Feature_M
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,3.833033,19148.159950,1.004217,110.828150,7.991267,4.046067
std,1.363423,10308.292904,0.255293,757.215597,38.160559,1.231131
min,1.700000,1930.200000,0.100000,0.000000,0.000000,0.100000
25%,2.700000,12074.275000,0.900000,0.000000,0.000000,4.000000
50%,3.700000,18134.000000,1.000000,0.000000,0.000000,4.000000
75%,4.700000,24082.325000,1.200000,0.000000,0.000000,4.500000
max,9.000000,103855.300000,1.600000,9999.900000,368.300000,9.900000


In [8]:
eval_df = pd.read_csv('./synthetic_dataset.csv')
eval_df.describe()

,Feature_A,Feature_C,Feature_E,Feature_K,Feature_L,Feature_M
count,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000,4500.000000
mean,3.865022,18963.022244,1.015200,104.466267,9.567244,4.062222
std,1.355890,10754.768972,0.252236,747.609124,41.795806,1.243912
min,1.700000,1921.400000,0.100000,0.000000,0.000000,0.100000
25%,2.800000,11596.125000,0.900000,0.000000,0.000000,4.000000
50%,3.700000,17723.300000,1.000000,0.000000,0.000000,4.000000
75%,4.700000,24023.625000,1.300000,0.000000,0.000000,4.500000
max,9.000000,122658.300000,1.600000,9999.900000,377.000000,9.900000


In [9]:
eval_df.head()

,Feature_A,Feature_B,Feature_C,Feature_D,Feature_E,Feature_F,Feature_G,Feature_H,Feature_I,Feature_J,Feature_K,Feature_L,Feature_M,Feature_N,Label
0,4.7,B_3,11475.4,D_2,0.9,F_2,G_1,H_2,I_1,J_1,0.0,0.0,4.0,N_1,Yes
1,4.2,B_3,16300.3,D_1,1.3,F_4,G_11,H_4,I_3,J_2,0.0,0.0,4.0,N_1,Yes
2,6.2,B_3,13631.4,D_6,1.0,F_2,G_1,H_3,I_1,J_2,0.0,0.0,4.0,N_1,Yes
3,1.8,B_3,7405.4,D_3,0.7,F_1,G_6,H_4,I_5,J_2,0.0,0.0,2.0,NaN,Yes
4,4.9,B_2,26356.8,D_1,1.3,F_2,G_2,H_2,I_1,J_1,0.0,0.0,6.0,N_1,No


In [11]:
pd.read_csv('./sample_submission.csv').head()

,entry_0,0.2
0,entry_1,0.2
1,entry_2,0.2
2,entry_3,0.2
3,entry_4,0.2
4,entry_5,0.2
